In [29]:
import numpy as np
import json
from enum import Enum

# Classes

In [30]:
# Convolution
class Convolution:
    def __init__(self, kernel, padding, stride, nb_filter, fct_activation):
        self.kernel = kernel
        self.padding = padding
        self.stride = stride
        self.nb_filter = nb_filter
        self.fct_activation = fct_activation

In [31]:
# input
class InputLayer:
    def __init__(self, shape):
        self.shape = shape

In [32]:
# Pooling Avg/Max
class Pooling:
    def __init__(self, op, kernel=2, padding="valid", stride=None):
        self.op = op # is the operation wanted (avg/max)
        self.kernel = kernel
        self.padding = padding
        self.stride = stride

In [33]:
# Class Flatten
class Flatten:
    pass

In [34]:
# Dense --> Fully connected layer
class Dense:
    def __init__(self, nb_neurones, fct_activation):
        self.nb_neurones = nb_neurones
        self.fct_activation = fct_activation

In [35]:
# identity block 
class IdBlock:
    def __init__(self, kernel, padding, nb_filter, fct_activation):
        self.kernel = kernel
        self.padding = padding
        self.nb_filter = nb_filter
        self.stride = 1
        self.fct_activation = fct_activation

In [36]:
# convolution block
class ConvBlock:
    def __init__(self, kernel, padding, nb_filter, stride, fct_activation):
        self.kernel = kernel
        self.nb_filter = nb_filter
        self.padding = padding
        self.stride = stride
        self.fct_activation = fct_activation

In [37]:
# Enum type archi
class TypeArchi(Enum):
    ALL = 0
    LENET = 1
    RESNET = 2
    DENSENET = 3

In [38]:
# configuration Archi
class ConfArchi:
    def __init__(self, type_archi, epsilon, dropout_rate, compress_factor):
        self.type_archi = type_archi
        self.epsilon = epsilon
        self.dropout_rate = dropout_rate
        self.compress_factor = compress_factor
    

In [39]:
# DenseNet Convolution Block 
class DenseNetBlock:
    def __init__(self, kernel, padding, nb_filter, stride, nb_layer, fct_activation, op, nb_block_densenet):
        self.kernel = kernel
        self.nb_filter = nb_filter
        self.padding = padding
        self.stride = stride
        self.nb_layer = nb_layer
        self.fct_activation = fct_activation
        self.op = op
        self.nb_block_densenet = nb_block_densenet

In [40]:
# Global pooling 
class GlobalPooling:
    def __init__(self, op):
        self.op = op    

# Functions

In [41]:
# read json file 
# return list of layer + graph
def readJsonFile(json_file):
    with open(json_file) as archi_json:
        file = json.load(archi_json)
    
    # instantiate class
    list_layer = []
    conf_archi = ""
    for layer in file:
        parameters = layer["parameters"] # get parameters
        # if is Input class
        if(layer["class"] == "InputLayer"):
            # instantiate class and add into the list
            list_layer.append(InputLayer(parameters["shape"]))

        
        # if is Pooling class
        elif(layer["class"] == "Pooling"):
               
            list_layer.append(Pooling(parameters["op"],
                                      parameters["kernel"],
                                      parameters["padding"],
                                      parameters["stride"]))
            
        # if is Convolution class
        elif(layer["class"] == "Convolution"):
                
                
            list_layer.append(Convolution(parameters["kernel"],
                                          parameters["padding"],
                                          parameters["stride"],
                                          parameters["nb_filter"],
                                          parameters["fct_activation"]))
            
        # if is Flatten class
        elif(layer["class"] == "Flatten"):
            list_layer.append(Flatten())
        
        # if is Dense class
        elif(layer["class"] == "Dense"):
            list_layer.append(Dense(parameters["nb_neurones"],
                                    parameters["fct_activation"]))
        
        elif(layer["class"] == "IdBlock"):
            list_layer.append(IdBlock(parameters["kernel"],
                                    parameters["padding"],
                                    parameters["nb_filter"],
                                    parameters["fct_activation"]))
            
        elif(layer["class"] == "ConvBlock"):
            list_layer.append(ConvBlock(parameters["kernel"],
                                        parameters["padding"],
                                        parameters["nb_filter"],
                                        parameters["stride"],
                                        parameters["fct_activation"]))
        elif(layer["class"] == "DenseNetBlock"):
            list_layer.append(DenseNetBlock(parameters["kernel"], 
                                            parameters["padding"],
                                            parameters["nb_filter"], 
                                            parameters["stride"], 
                                            parameters["nb_layer"],
                                            parameters["fct_activation"],
                                            parameters["op"],
                                            parameters["nb_block_densenet"]))
        elif(layer["class"] == "GlobalPooling"):
            list_layer.append(GlobalPooling(parameters["op"]))
        
        elif(layer["class"] == "ConfArchi"):
            conf_archi = ConfArchi(parameters["type_archi"], 
                                   parameters["epsilon"], 
                                   parameters["dropout_rate"], 
                                   parameters["compress_factor"]
                                  )

            
        else : print("Error")
    return list_layer, conf_archi


In [42]:
# return import of the py file
def writeImport():
    return """
import numpy as np
import os
from keras import backend as K
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D, MaxPool2D, Concatenate, Dropout
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.utils import plot_model
import tensorflow as tf
import sys
import traceback
import csv
from time import time

"""

In [43]:
def writeGlobalVariable(config):
    str_global = "\ntype_archi = '" + str(TypeArchi(config.type_archi).name) + "'\n"
    str_global += "epsilon = " + str(config.epsilon) + "\n"
    str_global += "dropout_rate = "+ str( config.dropout_rate) + "\n"
    str_global += "axis = 3\n"
    str_global += "compress_factor = "+ str(config.compress_factor) + "\n\n"
    
    return str_global

In [44]:
# write mnist data set
def writeMnistDataset():
    return """(train_x, train_y), (test_x, test_y) = keras.datasets.mnist.load_data()

# normaliser les pixel 0-255 -> 0-1
train_x = train_x / 255.0
test_x = test_x / 255.0

train_x = tf.expand_dims(train_x, 3)
test_x = tf.expand_dims(test_x, 3)

val_x = train_x[:5000]
val_y = train_y[:5000]

"""

In [45]:
# write cifar data set
def writecifar10Dataset():
    return """
# load dataset
(train_x, train_y), (test_x, test_y) = keras.datasets.cifar10.load_data()

# normalize to range 0-1
train_x = train_x / 255.0
test_x = test_x / 255.0

val_x = train_x[:5000]
val_y = train_y[:5000]
    
"""

In [46]:
# write function for identity block
def write_identity_block():
    return """
def id_block(X, f, filters, activation):

    X_shortcut = X

    X = Conv2D(filters=filters, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    if epsilon != 0:
        X = BatchNormalization(epsilon = epsilon, axis=axis)(X)
    X = Activation(activation)(X)


    X = Conv2D(filters=filters, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    if epsilon != 0:
        X = BatchNormalization(epsilon = epsilon, axis=axis)(X)

    X = Add()([X, X_shortcut])# SKIP Connection
    X = Activation(activation)(X)

    return X
    """

In [47]:
def write_conv_block():
    return"""
def conv_block(X, f, filters, activation, s=2):

    X_shortcut = X

    X = Conv2D(filters=filters, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X)
    if epsilon != 0:
        X = BatchNormalization(epsilon = epsilon, axis=axis)(X)
    X = Activation(activation)(X)

    X = Conv2D(filters=filters, kernel_size=(f, f), strides=(1, 1), padding='same', kernel_initializer=glorot_uniform(seed=0))(X)
    if epsilon != 0:
        X = BatchNormalization(epsilon = epsilon, axis=axis)(X)

    X_shortcut = Conv2D(filters=filters, kernel_size=(1, 1), strides=(s, s), padding='valid', kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    if epsilon != 0:
        X_shortcut = BatchNormalization(epsilon = epsilon, axis=axis)(X_shortcut)


    X = Add()([X, X_shortcut])
    X = Activation(activation)(X)

    return X
    """

In [48]:
def write_dense_block():
    return"""
def denseBlock(X, f, nb_filter, nb_layer, padding, activation):
    x_input = X    
    for _ in range(0,nb_layer):
        if epsilon != 0:
            X = BatchNormalization(epsilon = epsilon, axis=axis)(X)
        X = Activation(activation)(X)
        X = Conv2D(filters=nb_filter, kernel_size=(f, f), strides=(1, 1), padding=padding)(X)
        if dropout_rate != 0:
            X = Dropout(dropout_rate)(X)
    X = Concatenate()([X, x_input])
    return X
    """

In [49]:
def write_transition_block():
    return"""
def transition_block(X, f, nb_filter, padding, activation, op, stride):
    if epsilon != 0:
            X = BatchNormalization(epsilon = epsilon, axis=axis)(X)
    X = Activation(activation)(X)
    X = Conv2D(filters=nb_filter, kernel_size=(f, f), strides=(1, 1), padding=padding)(X)
    if dropout_rate != 0:
        X = Dropout(dropout_rate)(X)

    if (op == 'avg'):
        X = AveragePooling2D(pool_size = f, strides=stride, padding=padding)(X)
    else :
        X = MaxPooling2D(pool_size=f, strides=stride, padding=padding)(X)

    return X
    """

In [50]:
def write_init_value():
    return """

# init training time
training_time = 0
# init result test/train
test_result_loss = ""
test_result_acc = ""

train_result_loss = ""
train_result_acc = ""

nb_layers = "not build"

"""

In [51]:
# return string of the wanted layer
def write_layer(layer):
    
    # if is Intput
    if isinstance(layer, InputLayer):
        return "        X_input = X = Input({})\n".format(
                layer.shape)
        
            
    # if is Convolution
    elif isinstance(layer, Convolution):
        return "        X = Conv2D({}, kernel_size={}, strides={}, activation='{}', padding='{}')(X)\n".format(
                layer.nb_filter,
                layer.kernel,
                layer.stride,
                layer.fct_activation,
                layer.padding)
            
    # if is Pooling
    elif isinstance(layer, Pooling):
        if(layer.op == "avg"): # avg Pooling 
            return "        X = AveragePooling2D(pool_size={}, strides={}, padding='{}')(X)\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
                
        else : # Max Pooling
            return  "        X = MaxPooling2D(pool_size={}, strides={}, padding='{}')(X)\n".format(
                    layer.kernel,
                    layer.stride,
                    layer.padding)
    elif isinstance(layer, IdBlock):
        return "        X = id_block(X, {}, {}, '{}')\n".format(layer.kernel, layer.nb_filter, layer.fct_activation)
    elif isinstance(layer, ConvBlock):
        return "        X = conv_block(X, {}, {}, '{}', {})\n".format(layer.kernel, layer.nb_filter, layer.fct_activation, layer.stride)
    elif isinstance(layer, DenseNetBlock):
        str_dense_block = ""
        for nb in range(0,layer.nb_block_densenet):
            str_dense_block += "        X = denseBlock(X, {}, {}, {}, '{}', '{}')\n".format(
                layer.kernel, 
                layer.nb_filter,
                layer.nb_layer,
                layer.padding,
                layer.fct_activation
            )
        
        str_dense_block += "        X = transition_block(X, {}, {}, '{}', '{}', '{}', {})\n".format(
            layer.kernel, 
            layer.nb_filter, 
            layer.padding,
            layer.fct_activation,
            layer.op,
            layer.stride
        )
        return str_dense_block
    elif isinstance(layer, GlobalPooling):
        if(layer.op == "avg"):
            return "        X = GlobalAveragePooling2D()(X)\n"
        else:
            return "        X = GlobalMaxPooling2D()(X)\n"
    
    elif isinstance(layer, Flatten):
        return "        X = Flatten()(X)\n"
    elif isinstance(layer, Dense):
        return "        X = Dense({}, activation='{}')(X)\n".format(
                        layer.nb_neurones, 
                        layer.fct_activation)
    # Not possible
    else : print("Not Possible")

In [52]:
def create_CNN(archi):
    
    str_model_cnn = "def getModel():\n"
    
    for l in archi:
        str_model_cnn += write_layer(l)
    
    str_model_cnn += "        model = Model(inputs=X_input, outputs=X)\n"
    str_model_cnn += "        return model\n\n"
    str_model_cnn += "    model = getModel()\n"
    return str_model_cnn

In [53]:
def create_py_file(json_file):
    s = json_file.split(".")
    file_name = s[0].split("/")[1]
    architecture, config = readJsonFile(json_file)
    
    type_archi = TypeArchi(config.type_archi)
    
    print(type_archi)
    
    # python directory
    py_dir = "architecture_py/"
    
    # log directory
    log_dir = "../architecture_log/"
    
    # png directory
    png_dir = "../architecture_img/"
    
    # reset file
    file_py = open(py_dir + file_name + ".py", "w")
    file_py.close()
    
    file_py = open(py_dir + file_name + ".py", "a") # Open file in writting (a --> append)
    
    # write import
    file_py.write(writeImport())
    
    #write global variable
    file_py.write(writeGlobalVariable(config))
    
    # write train/test data 
    file_py.write(writecifar10Dataset())
    
    # write init value 
    file_py.write(write_init_value())
    
    # add function
    if (type_archi == TypeArchi.ALL):
        file_py.write(write_identity_block())
        file_py.write(write_conv_block())
        file_py.write(write_dense_block())
        file_py.write(write_transition_block())
    
    elif (type_archi == TypeArchi.DENSENET):
        file_py.write(write_dense_block())
        file_py.write(write_transition_block())
    
    elif (type_archi == TypeArchi.RESNET):
        file_py.write(write_identity_block())
        file_py.write(write_conv_block())
        
    else:
        pass
    
    # try
    file_py.write("""
try:
    """)
    
    # write architecture model
    file_py.write(create_CNN(architecture,))
        
    # write : create png of the model
    file_py.write("    plot_model(model, show_shapes=True, to_file=\"%s\")\n" % (png_dir+file_name+".png"))
    
    # write compiler
    file_py.write("""    model.compile(optimizer='adam', loss=keras.losses.sparse_categorical_crossentropy, metrics=['accuracy'])\n\n""")
    
    # write way for time computation
    file_py.write("""    start = time()\n""")
    
    # write callbacks function
    file_py.write("""    es = tf.keras.callbacks.EarlyStopping(monitor='loss', verbose=1)
    list_cb = [es]\n""")
    
    # write model training
    file_py.write("""    history = model.fit(train_x, train_y, epochs=50, validation_split=0.1, callbacks=list_cb)\n""")
    
    # write time computation
    file_py.write("""    training_time = time()-start\n""")
    
    # write model evaluation
    file_py.write("""    print(model.evaluate(test_x, test_y))\n""")
    
    
    # all is great
    log_file = log_dir + file_name +".log"
    file_py.write("""
    log_file = open(\"""" + log_file + """\" , "w")
    
    # save test result
    log_file.write('test result : ' + str(model.evaluate(test_x, test_y)))
    test_result_loss = model.evaluate(test_x, test_y)[0]
    test_result_acc = model.evaluate(test_x, test_y)[1]
    
    # save train result
    log_file.write('train result : ' + str(model.evaluate(test_x, test_y)))
    log_file.write('History train result : ' + str(history.history))
    train_result_loss = model.evaluate(train_x, train_y)[0]
    train_result_acc = model.evaluate(train_x, train_y)[1]
    
    print('OK: file """ + log_file +""" has been create')
    
    nb_layers = len(model.layers)
    log_file.close()
""")
    
    # something go wrong 
    error_file = log_dir + file_name + "_error.log"
    file_py.write("""except:
    print('error: file """ + error_file +""" has been create')
    error_file = open(\"""" + error_file + """\" , "w")
    traceback.print_exc(file=error_file)
    result_loss = "Error"
    result_acc = "Error"
    error_file.close()
""")
    
    file_py.write("""finally:
    file = open('../architecture_results_v3.csv', 'a', newline ='')
    with file: 

        # identifying header   
        header = ['file_name', 'training_time(s)', 'test_result_loss', 'test_result_acc', 'train_result_acc', 'train_result_loss', 'nb_layers', 'epochs', 'type_archi'] 
        writer = csv.DictWriter(file, fieldnames = header) 
      
        # writing data row-wise into the csv file 
        # writer.writeheader() 
        writer.writerow({'file_name' : '"""+ file_name + """',  
                         'training_time(s)': training_time,  
                         'test_result_loss': test_result_loss,
                         'test_result_acc': test_result_acc,
                         'train_result_acc': train_result_acc,
                         'train_result_loss': train_result_loss,
                         'nb_layers': nb_layers,
                         'epochs' : len(history.history['loss']),
                         'type_archi': type_archi}) 
        print('add line into architecture_results_v3.csv')
    file.close()
    """)
    
    
    
    # close
    file_py.close()
    

# Test

In [54]:
directory = "architecture_json/"
archi, conf_archi = readJsonFile(directory+"archi_V3_test.json")

In [55]:
archi

# Create Py file

In [56]:
directory = "architecture_json/"
for i in range(1,101):
    file_name = "archi_v3_%d.json" % i
    print(directory+file_name)
    create_py_file(directory+file_name)
    print("")

architecture_json/archi_v3_1.json
TypeArchi.ALL

architecture_json/archi_v3_2.json
TypeArchi.ALL

architecture_json/archi_v3_3.json
TypeArchi.ALL

architecture_json/archi_v3_4.json
TypeArchi.ALL

architecture_json/archi_v3_5.json
TypeArchi.ALL

architecture_json/archi_v3_6.json
TypeArchi.ALL

architecture_json/archi_v3_7.json
TypeArchi.ALL

architecture_json/archi_v3_8.json
TypeArchi.ALL

architecture_json/archi_v3_9.json
TypeArchi.ALL

architecture_json/archi_v3_10.json
TypeArchi.ALL

architecture_json/archi_v3_11.json
TypeArchi.ALL

architecture_json/archi_v3_12.json
TypeArchi.ALL

architecture_json/archi_v3_13.json
TypeArchi.ALL

architecture_json/archi_v3_14.json
TypeArchi.ALL

architecture_json/archi_v3_15.json
TypeArchi.ALL

architecture_json/archi_v3_16.json
TypeArchi.ALL

architecture_json/archi_v3_17.json
TypeArchi.ALL

architecture_json/archi_v3_18.json
TypeArchi.ALL

architecture_json/archi_v3_19.json
TypeArchi.ALL

architecture_json/archi_v3_20.json
TypeArchi.ALL

architect